In [41]:

import sqlalchemy as db
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pytz import timezone, utc
from new_api import (add_entity, create_new_entity_set, add_aggregation_features, get_training_df, get_prediction_df, 
add_entity_df, list_features, generate_base_model_definition)
from ludwig.api import LudwigModel


In [42]:
agent_entity = {"name": "agent", "table": "agent", "type": "primary", "index": "agent_id", "time": {"field": "effective_date", "type": "effective_date"}}
add_entity(agent_entity)


acxiom_entity = {"name": "acxiom", "table": "agent_acxiom", "type": "primary", "index": "agent_id", "time": {"field": "date", "type": "effective_date"}}
add_entity(acxiom_entity)


comission_events = {"name": "agent_commission", "table": "agent_sales", "type": "event", 'index': "id", "time": {"field": "date", "type": "event"}}
add_entity(comission_events)



In [43]:
relationships =  [
            ("one_to_one", {"name": "agent", "index": "agent_id"}, {"name": "acxiom", "index": "agent_id"}),
            ("one_to_many", {"name": "agent", "index": "agent_id"}, {"name": "agent_commission",  "index": "agent_id"})
                ]


create_new_entity_set(name="nyl_agents", entities=["agent", "acxiom", "agent_commission"], relationships=relationships)


In [44]:
agent_commission_agg_features = {"total_sales": {"feature": "amount","function":"sum", "name": "total_sales", "time_window": "full_history"},
                             "max_sales":  {"feature": "amount", "function":"max", "name": "max_sales", "time_window": "full_history"},
                             "total_num_sales":   {"feature": "id", "function": "count", "name": "total_num_sales", "time_window": "full_history"}
                            }
add_aggregation_features("agent_commission", agent_commission_agg_features)


In [36]:
available_features = list_features("nyl_agents")
available_features

{'agent': {'raw_features': ['index',
   'effective_date',
   'agent_id',
   'feature_1',
   'feature_2',
   'feature_3',
   'feature_4',
   'feature_5',
   'feature_6',
   'feature_7',
   'feature_8']},
 'acxiom': {'raw_features': ['index',
   'date',
   'agent_id',
   'zipcode',
   'num_household']},
 'agent_commission': {'raw_features': ['index',
   'id',
   'date',
   'agent_id',
   'amount',
   'feature_2',
   'feature_3'],
  'calulated_features': ['total_sales', 'max_sales', 'total_num_sales']}}

In [45]:
days = [datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0).replace(tzinfo=utc) \
        - timedelta(day * 365) for day in range(2)][::-1]

agents = [1001, 1002, 1003, 1004, 1005]


df = pd.DataFrame(
    {
        "observation_time": [day for day in days for customer in agents],
        "agent_id": [customer for day in days for customer in agents],
        "prediction": [np.random.rand()  for _ in range(len(days) * len(agents))],

    }
)


eol = {"pk": "agent_id", "observation_date": "observation_time", "label": "prediction"}

In [46]:
features = {"entity_set": "nyl_agents",
            "target_entity": "agent",
            "features": {
                         "agent": ["feature_1", "feature_2", "feature_6"],
                         "acxiom": ["zipcode", "num_household"],
                         "agent_commission": ["total_sales", "max_sales"]

                    },
            "observations": {"type": "eol", "eol": eol, "data": df}
           
            }
training_df = get_training_df(features)
training_df

(builtins.AttributeError) 'NoneType' object has no attribute 'cursor'
[SQL: DESCRIBE `nyl_agents_eol`]


StatementError: (builtins.AttributeError) 'NoneType' object has no attribute 'cursor'
[SQL: select nyl_agents_eol.agent_id, nyl_agents_eol.observation_time, nyl_agents_eol.prediction, agent1.feature_1, agent1.feature_2, agent1.feature_6, agent_acxiom1.zipcode, agent_acxiom1.num_household , agent_salestotal_sales1.total_sales , agent_salesmax_sales1.max_sales from nyl_agents_eol  left join agent agent1 on nyl_agents_eol.agent_id = agent1.agent_id and           agent1.effective_date = (select max(effective_date) from agent agent2 where agent2.agent_id = nyl_agents_eol.agent_id and agent2.effective_date <= nyl_agents_eol.observation_time) left join agent_acxiom agent_acxiom1 on nyl_agents_eol.agent_id = agent_acxiom1.agent_id and           agent_acxiom1.date = (select max(date) from agent_acxiom agent_acxiom2 where agent_acxiom2.agent_id = nyl_agents_eol.agent_id and agent_acxiom2.date <= nyl_agents_eol.observation_time) left join (select nyl_agents_eol.agent_id, nyl_agents_eol.observatio

In [8]:
prediction_df = get_prediction_df(features)
prediction_df


,agent_id,feature_1,feature_2,feature_6,zipcode,num_household,total_sales,max_sales
0,1001,8.676654,6.617975,3.793197,0.189578,0.180422,7763.183214,99.683471
1,1002,0.619986,1.829505,4.545613,8.482659,8.792610,7620.647214,99.829563
2,1003,3.943603,5.452559,2.133420,8.535475,2.040803,8194.756111,99.406206
3,1004,0.795607,6.487110,0.468483,3.698678,5.276331,7870.330874,99.753397
4,1005,5.625281,2.807297,5.041419,9.196020,3.594730,8326.790783,99.512503


In [9]:
features2 = {"entity_set": "nyl_agents",
            "target_entity": "agent_commision",
            "features": {
                         "agent": ["feature_1", "feature_2", "feature_6"],
                         "acxiom": ["zipcode", "num_household"],
                         "agent_commission": ["amount", "date", "feature_2", "feature_3", "total_sales", "max_sales"]

                    },
            "observations": {"type": "event"}
            }
#training_df = get_training_df(features
#training_df

In [10]:
from ludwig.api import LudwigModel

In [20]:
generate_base_model_definition(features)

int64
datetime64[ns]
float64
float64
float64
float64
float64
float64
float64


In [ ]:
# %load model_definition.yaml
input_features:
- name: agent_id
  type: numerical
- name: observation_time
  type: numerical
- name: feature_1
  type: numerical
- name: feature_2
  type: numerical
- name: feature_6
  type: numerical
- name: zipcode
  type: numerical
- name: num_household
  type: numerical
- name: total_sales
  type: numerical
- name: max_sales
  type: numerical
output_features:
- name: prediction
  type: numerical


In [26]:
%%writefile model_definition.yaml
input_features:
- name: agent_id
  type: numerical
- name: feature_1
  type: numerical
- name: feature_2
  type: numerical
- name: feature_6
  type: numerical
- name: zipcode
  type: numerical
- name: num_household
  type: numerical
- name: total_sales
  type: numerical
- name: max_sales
  type: numerical
output_features:
- name: prediction
  type: numerical

Overwriting model_definition.yaml


In [27]:
ludwig_model = LudwigModel(model_definition_file='model_definition.yaml')
train_status = ludwig_model.train(data_df=training_df)